In [ ]:
"""
This example shows how to define your own dataset and use it to train a
non-trivial GNN with message-passing and pooling layers.
The script also shows how to implement fast training and evaluation functions
in disjoint mode, with early stopping and accuracy monitoring.
The dataset that we create is a simple synthetic task in which we have random
graphs with randomly-colored nodes. The goal is to classify each graph with the
color that occurs the most on its nodes. For example, given a graph with 2
colors and 3 nodes:
x = [[1, 0],
     [1, 0],
     [0, 1]],
the corresponding target will be [1, 0].
"""

import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import os

!pip install spektral
import spektral
from spektral.data import Dataset, DisjointLoader, Graph, BatchLoader
from spektral.layers import GCSConv, GlobalAvgPool
from spektral.transforms.normalize_adj import NormalizeAdj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 14.2 MB/s 


In [ ]:
from spektral.datasets import TUDataset
dataset2 = TUDataset('PROTEINS')
print(dataset2)
print(dataset2[100])

Successfully loaded PROTEINS.
TUDataset(n_graphs=1113)
Graph(n_nodes=50, n_node_features=4, n_edge_features=None, n_labels=2)


In [ ]:
path_of_the_directory = '/content/drive/MyDrive/reaxff/xmolout'
ext = ('')
for files in sorted(os.listdir(path_of_the_directory)): #now sequential
    if files.endswith(ext):
        #print(files)
        f2 = files  
    else:
        continue

In [ ]:
def remove(string):
    return string.replace(" ", "")

def distance_iter_mat(x, d_min, d_max, a1_index, a2_index):

  #a1 index = line no of 1st atom in xmolout
  #a2 index = line no of 2nd atom in xmolout
  
  x_a1 = np.zeros(math.ceil(len(x)/76))
  y_a1 = np.zeros(math.ceil(len(x)/76))
  z_a1 = np.zeros(math.ceil(len(x)/76))
  x_a2 = np.zeros(math.ceil(len(x)/76))
  y_a2 = np.zeros(math.ceil(len(x)/76))
  z_a2 = np.zeros(math.ceil(len(x)/76))


  for i_x in range (0, math.floor(len(x)/76)):
  #print(i_x)
  #print(x[9+76*i_x])
  #print(x[9+76*i_x][4:12]) 
    x_a1[i_x] = float(x[a1_index+76*i_x][4:12])
    x_a2[i_x] = float(x[a2_index+76*i_x][4:12])

  for i_y in range (0, math.floor(len(x)/76)):
  #print(i_y)
  #print(x[9+76*i_y])
  #print(x[9+76*i_y][13:21]) 
    y_a1[i_y] = float(x[a1_index+76*i_y][13:21])
    y_a2[i_y] = float(x[a2_index+76*i_y][13:21])
  for i_z in range (0, math.floor(len(x)/76)):
  
  
  #print(x[9+76*i_z][13:21]) 
    z_a1[i_z] = float(x[a1_index+76*i_z][22:31])
    z_a2[i_z] = float(x[a2_index+76*i_z][22:31])




    x_diff = np.square(x_a1-x_a2)
    y_diff = np.square(y_a1-y_a2)
    z_diff = np.square(z_a1-z_a2)
    dist_a1_a2 = np.sqrt(x_diff + y_diff + z_diff)

  #print(dist_a1_a2)

  res_a1_a2 = np.where(np.logical_and(dist_a1_a2 >= d_min, dist_a1_a2 <= d_max))
  #print(res_a1_a2)
  print(type(res_a1_a2))
  return res_a1_a2#, dist_a1_a2

  #for i_ts in res_c1_n1:
    #print(i_ts)
    #print(dist_a1_a2[i_ts])



def common_member(a, b):
    a_set = set(a)
    b_set = set(b)
 
    if (a_set & b_set):
        print(a_set & b_set)
    else:
        print("No common elements")

def distance_check(dist_matrix, d_min, d_max): #not checking others as they are molecules themselves
 

  #dist_matrix -> the matrix calculated by dist function above
  c1_line = 59 #line nummer in xmolout counting from 0 onwards
  n1_line = 9
  o1_line = 60
  h1_line_1 = 20  #both h connected to n can react
  h1_line_2 = 21
  x_r = np.zeros(100,15,100)
  for i_dm in range(0, len(dist_matrix)):
    x_r = np.where(np.logical_and(dist_matrix >= d_min, dist_matrix <= d_max))
  
def contains(tuple, given_char):
    if given_char in tuple:
        print(given_char)
        return True
    return False   
  

def common(a,b): 
  c = [value for value in a if value in b] 
  return c

    

In [ ]:
path_of_the_directory_2 = '/content/drive/MyDrive/reaxff/tracking'
ext = ('')

trk_files = sorted(os.listdir(path_of_the_directory_2))
#print(trk_files)

path_of_the_directory_3 = '/content/drive/MyDrive/reaxff/molfra'

molfras = sorted(os.listdir(path_of_the_directory_3))
#print(molfras)

In [ ]:
import math

for files in sorted(os.listdir(path_of_the_directory)): #now sequential
  i_track = 0
  path = '/content/drive/MyDrive/reaxff/xmolout' + '/' + files
  fp = open(path, 'r')
  x = fp.readlines()
  #print(x[9])
  path_tk = '/content/drive/MyDrive/reaxff/tracking' + '/' + trk_files[i_track]
  i_track = i_track + 1
  ft = open(path_tk, 'r')
  track = ft.readlines()
  #print(track)
  path_mol = '/content/drive/MyDrive/reaxff/molfra' + '/' + molfras[i_track]
  fm = open(path_mol, 'r')
  molf = fm.readlines()

  r1 = list(distance_iter_mat(x, 3, 12, 9, 59))
#9 is for n1 and 59 for c1
# n1 - 10th - 9 in xmolout
# c1 - 58 - 59
# h1 - 20/21 - 19/20
# o1 - 59 - 60
#numbering in xmolout starts from zero
  #print((track[2][41:45]))
  #print(track[2][49:54])


#  distance_iter_mat(x,float(remove(track[2][41:45])), float(remove(track[2][49:54])), 60, 61)  #o1 and c1
  
  #print(type(intersection_1))
  #print(x[28])
  r2 = list(distance_iter_mat(x, float(remove(track[2][65:69])), float(remove(track[2][70:75])), 28, 60))   #h1 and o1
  #print('r1/r2', r1,r2)

  for r1_e in r1[0]:
    for r2_e in r2[0]:
      if r1_e == r2_e:
        print('intersection of nc and oh', r1_e, r2_e)
  #index_of_mofra_rxn = molf[i_track]
  #print(molf)


  mylist = ['abc123', 'def456', 'ghi789']
  sub = '178.1440'
  next((s for s in molf if sub in s), None) # returns 'abc123'

In [ ]:
#Printing what kind of reaction happens in each of the cases using their molfra file
#127 reactions total 

Reac_outcome = np.zeros((1,500))
i_reac = 1
for files_m in sorted(os.listdir('/content/drive/MyDrive/reaxff/molfra')):
  path_m = '/content/drive/MyDrive/reaxff/molfra' + '/' + files_m
  fm = open(path_m, 'r')
  x_mo = fm.readlines()
  #print('len = ', len(x_mo))
  for i_xmo in range(3,4):#len(x_mo))):
    #print(files_m)
    #print(i_reac)

    #print(x_mo[len(x_mo)-3][27:])
    if int(remove((x_mo[len(x_mo)-3][27:]))) > 2:
      #print(files_m, 'Rxn happened, stuff broken down')
      Reac_outcome[0][i_reac] = 1
    elif int(remove((x_mo[len(x_mo)-3][27:]))) == 1:
      #print(files_m, 'Rxn happened, molecules joined')
      Reac_outcome[0][i_reac] = 1
    elif int(remove((x_mo[len(x_mo)-3][27:]))) == 2:
      #print(x_mo[2])
      #print(x_mo[len(x_mo)-5])
      if (x_mo[2][10:]) == (x_mo[len(x_mo)-5][10:]):
        print(files_m, 'No Rxn')
      else:
        print('Miscellaneous Rxn') 
    i_reac = i_reac + 1

In [ ]:
print(Reac_outcome[:100])

[[0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
  0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
  0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 

In [ ]:
def input_proces(atom_id, path_geo):
  fg = open(path_geo, 'r')
  x_fg = fg.readlines()
  inp1 = np.zeros((1,4))
  inp1[0][0] = ord(x_fg[atom_id][13])#atom 
  inp1[0][1] = float(remove(x_fg[atom_id][30:41])) # x-coord
  inp1[0][2] = float(remove(x_fg[atom_id][41:52]))
  inp1[0][3] = float(remove(x_fg[atom_id][52:61]))
  print(inp1)
  return(inp1)

In [ ]:
#read_attr('/content/drive/MyDrive/reaxff/geo/geo_002')
geo_files = sorted(os.listdir('/content/drive/MyDrive/reaxff/geo'))
print(geo_files)

mol_1_attr = np.zeros((500, 3, 4))
mol_2_attr = np.zeros((500, 3, 4))

for i_g in range(1,500):
  #if 2*i_g%2 == 1:
   # 2*i_g = 2*i_g - 1

  path_geo_even = geo_files[2*i_g]
  #print(i_g, 2*i_g, path_geo_even)
  #mol_1_attr = np.zeros((3,4))
  for atom_id in [63,64,65]:#,14,26,27]:
    path_geo_even = '/content/drive/MyDrive/reaxff/geo' + '/' + geo_files[2*i_g]
    print(path_geo_even)
    mol_1_attr[i_g][atom_id-63] = input_proces(atom_id, path_geo_even)#'/content/drive/MyDrive/reaxff/geo/geo_002')

  #mol_2_attr = np.zeros((3,4))
  i_1 = 0
  for atom_id in [14,26,27]:
    
    path_geo_even = '/content/drive/MyDrive/reaxff/geo' + '/' + geo_files[2*i_g]
    print(path_geo_even)
    mol_2_attr[i_g][i_1] = input_proces(atom_id, path_geo_even)#'/content/drive/MyDrive/reaxff/geo/geo_002')
    i_1 = i_1 + 1

In [ ]:
#CONSTANT CONNECTIVITY MATRIX (from slide 10)

conn_mat_1 = [[0,1,1],[1,0,0],[1,0,0]] #nh2
conn_mat_2 = [[0,1,1],[1,0,1],[1,1,0]] #cco


In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, nodes, feats, **kwargs):
        self.nodes = nodes
        self.feats = feats
        print('run1')
        super().__init__(**kwargs)


    # Create the directory
        #print(self.path)
      #os.mkdir('/content/sample_data/custom_dataset' + str(self))

    # Write the data to file
        #for i0 in range(0,249):
         # x = mol_1_attr[i0]
          #print(mol_1_attr[i0])
          #a = [[0,1,1],[1,0,0],[1,0,0]]
          #y = Reac_outcome[0][i0]
          #print(y)
          #print('run2')
          #filename = os.path.join('/content/sample_data/Output', f'graph_{i0}')
          #np.savez(filename, x=x, a=a, y=y)

    def read(self):
    # We must return a list of Graph objects
      output = []

      for i01 in range(0,100):
        data = np.load(os.path.join('/content/sample_data/Output', f'graph_{i01}.npz'))
        output.append(
            Graph(x=data['x'], a=data['a'], y=data['y'])
        )

      return output

In [ ]:
for i0 in range(0,100):
          x = mol_1_attr[i0]
          print(mol_1_attr[i0])
          a = [[0,1,1],[1,0,0],[1,0,0]]
          y = np.zeros((4,))
          color_counts = x.sum(0)
          y[np.argmax(color_counts)] = 1#Reac_outcome[0][i0]
          print(y)
          print('run2')
          filename = os.path.join('/content/sample_data/Output', f'graph_{i0}')
          np.savez(filename, x=x, a=a, y=y)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[1. 0. 0. 0.]
run2
[[67.       9.04602  5.72952 16.39124]
 [67.       7.80473  5.14068 16.94526]
 [79.       8.77968  5.72713 17.77517]]
[1. 0. 0. 0.]
run2
[[67.       7.4314   0.44142  1.72934]
 [67.       6.69779  1.07713  2.84835]
 [79.       6.28321  1.22896  1.51109]]
[1. 0. 0. 0.]
run2
[[67.       5.2962   6.22972  2.53243]
 [67.       6.07721  6.8305   3.63859]
 [79.       6.05684  5.43982  3.41769]]
[1. 0. 0. 0.]
run2
[[67.       4.56899 12.69411 17.77566]
 [67.       3.14911 12.50895 18.15536]
 [79.       4.06808 13.14179 19.01455]]
[1. 0. 0. 0.]
run2
[[67.      19.17142  9.10437  1.64582]
 [67.      18.78649  9.0185   3.07374]
 [79.      18.64142  7.92256  2.20134]]
[1. 0. 0. 0.]
run2
[[67.      20.21404 11.94374  9.55411]
 [67.      20.62679 10.5394   9.78204]
 [79.      19.3506  10.8635   9.28256]]
[1. 0. 0. 0.]
run2
[[67.       6.08347 15.19488 14.50082]
 [67.       6.64574 13.84666 14.25453]
 [79.       5.63189 14.06244 15.207

In [ ]:
dataset = MyDataset(3, 4)

run1


In [ ]:
print(dataset)
print(dataset[43])
print(dataset[43].n_labels)
print(dataset2[43].n_labels)
print(dataset2[430].n_labels)

MyDataset(n_graphs=100)
Graph(n_nodes=3, n_node_features=4, n_edge_features=None, n_labels=4)
4
2
2


In [ ]:
max_degree = dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)
print(max_degree)

2


In [ ]:
from spektral.transforms import GCNFilter
dataset.apply(GCNFilter())

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [ ]:
class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [ ]:
model = MyFirstGNN(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

TypeError: ignored

In [ ]:
dataset_train = dataset[:90]
dataset_test = dataset[91:]

In [ ]:
from spektral.data import BatchLoader

loader = BatchLoader(dataset_train, batch_size=2)

In [ ]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0000e+00
Epoch 2/10
45/45 [==============================] - 0s 2ms/step - loss: 0.0000e+00
Epoch 3/10
 1/45 [..............................] - ETA: 0s - loss: 0.0000e+00

/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


45/45 [==============================] - 0s 2ms/step - loss: 0.0000e+00
Epoch 4/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0000e+00
Epoch 5/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0000e+00
Epoch 6/10
45/45 [==============================] - 0s 2ms/step - loss: 0.0000e+00
Epoch 7/10
45/45 [==============================] - 0s 2ms/step - loss: 0.0000e+00
Epoch 8/10
45/45 [==============================] - 0s 2ms/step - loss: 0.0000e+00
Epoch 9/10
45/45 [==============================] - 0s 2ms/step - loss: 0.0000e+00
Epoch 10/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0000e+00


In [ ]:
from spektral.data import BatchLoader

loader = BatchLoader(dataset_test, batch_size=32)

loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

1/1 [==============================] - 0s 124ms/step - loss: 0.0000e+00
Test loss: 0.0
